In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn 
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
cab_data = pd.read_csv(r'C:\Users\Asus\Downloads\cab_rides.csv', encoding= 'utf-8')
weather_data = pd.read_csv(r'C:\Users\Asus\Downloads\weather.csv', encoding= 'utf-8')

In [3]:
cab_data.describe()

,distance,time_stamp,price,surge_multiplier
count,693071.000000,6.930710e+05,637976.000000,693071.000000
mean,2.189430,1.544046e+12,16.545125,1.013870
std,1.138937,6.892342e+08,9.324359,0.091641
min,0.020000,1.543200e+12,2.500000,1.000000
25%,1.280000,1.543440e+12,9.000000,1.000000
50%,2.160000,1.543740e+12,13.500000,1.000000
75%,2.920000,1.544830e+12,22.500000,1.000000
max,7.860000,1.545160e+12,97.500000,3.000000


In [4]:
weather_data.describe()

,temp,clouds,pressure,rain,time_stamp,humidity,wind
count,6276.000000,6276.000000,6276.000000,894.000000,6.276000e+03,6276.000000,6276.000000
mean,39.090475,0.677777,1008.445209,0.057652,1.543857e+09,0.763985,6.802812
std,6.022055,0.314284,12.870775,0.100758,6.659340e+05,0.127340,3.633466
min,19.620000,0.000000,988.250000,0.000200,1.543204e+09,0.450000,0.290000
25%,36.077500,0.440000,997.747500,0.004900,1.543387e+09,0.670000,3.517500
50%,40.130000,0.780000,1007.660000,0.014850,1.543514e+09,0.760000,6.570000
75%,42.832500,0.970000,1018.480000,0.060925,1.544691e+09,0.890000,9.920000
max,55.410000,1.000000,1035.120000,0.780700,1.545159e+09,0.990000,18.180000


In [5]:
cab_data.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1.543370e+12,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1.543550e+12,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1.543460e+12,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [6]:
weather_data.head()

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49


In [7]:
cab_data['date_time'] = pd.to_datetime(cab_data['time_stamp']/1000, unit='s')
weather_data['date_time'] = pd.to_datetime(weather_data['time_stamp'], unit='s')

In [8]:
cab_data.head(2)

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time
0,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 08:46:40
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 00:53:20


In [9]:
weather_data.head(2)

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind,date_time
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25,2018-12-16 23:45:01
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32,2018-12-16 23:45:01


In [10]:
cab_data['merge_date'] = cab_data.source.astype(str) + " - " + cab_data.date_time.dt.date.astype(str) + \
" - " + cab_data.date_time.dt.hour.astype(str)
weather_data['merge_date'] = weather_data.location.astype(str) + " - " + \
weather_data.date_time.dt.date.astype(str) + " - " + weather_data.date_time.dt.hour.astype(str)

In [11]:
cab_data.head(2)

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time,merge_date
0,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018-12-16 08:46:40,Haymarket Square - 2018-12-16 - 8
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018-11-27 00:53:20,Haymarket Square - 2018-11-27 - 0


In [12]:
weather_data.index = weather_data['merge_date']

In [13]:
merged_data = cab_data.join(weather_data, on = 'merge_date', rsuffix='_w')

In [14]:
merged_data.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,...,temp,location,clouds,pressure,rain,time_stamp_w,humidity,wind,date_time_w,merge_date_w
0,0.44,Lyft,1.544950e+12,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,...,39.36,Haymarket Square,0.39,1022.44,NaN,1.544950e+09,0.74,8.14,2018-12-16 08:45:01,Haymarket Square - 2018-12-16 - 8
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,...,43.94,Haymarket Square,1.00,1006.29,0.0425,1.543278e+09,0.90,10.12,2018-11-27 00:17:14,Haymarket Square - 2018-11-27 - 0
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,...,43.76,Haymarket Square,0.97,1005.94,0.1390,1.543279e+09,0.92,11.22,2018-11-27 00:42:13,Haymarket Square - 2018-11-27 - 0
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,...,43.79,Haymarket Square,0.98,1006.14,0.2122,1.543280e+09,0.91,9.79,2018-11-27 00:45:21,Haymarket Square - 2018-11-27 - 0
1,0.44,Lyft,1.543280e+12,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,...,43.76,Haymarket Square,0.97,1005.93,0.2318,1.543279e+09,0.92,11.23,2018-11-27 00:42:27,Haymarket Square - 2018-11-27 - 0


In [15]:
merged_data['rain'].fillna(0,inplace=True)

In [16]:
merged_data = merged_data[pd.notnull(merged_data['date_time_w'])]

In [17]:
merged_data = merged_data[pd.notnull(merged_data['price'])]

In [18]:
merged_data['day'] = merged_data.date_time.dt.dayofweek

In [19]:
merged_data['hour'] = merged_data.date_time.dt.hour

In [20]:
merged_data.groupby('cab_type').count()

,distance,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,date_time,...,clouds,pressure,rain,time_stamp_w,humidity,wind,date_time_w,merge_date_w,day,hour
cab_type,,,,,,,,,,,,,,,,,,,,,
Lyft,560353,560353,560353,560353,560353,560353,560353,560353,560353,560353,...,560353,560353,560353,560353,560353,560353,560353,560353,560353,560353
Uber,603067,603067,603067,603067,603067,603067,603067,603067,603067,603067,...,603067,603067,603067,603067,603067,603067,603067,603067,603067,603067


In [21]:
X = merged_data[merged_data.product_id=='lyft_line'][['day','distance','hour','temp','clouds', 'pressure','humidity', 'wind', 'rain']]
y = merged_data[merged_data.product_id=='lyft_line']['price'] 

In [22]:
X.count()

day         93132
distance    93132
hour        93132
temp        93132
clouds      93132
pressure    93132
humidity    93132
wind        93132
rain        93132
dtype: int64

In [23]:
X.reset_index(inplace=True)
X = X.drop(columns=['index'])

In [24]:
X.head()

,day,distance,hour,temp,clouds,pressure,humidity,wind,rain
0,6,0.44,8,39.36,0.39,1022.44,0.74,8.14,0.0000
1,0,1.08,6,44.85,0.89,1000.70,0.95,1.53,0.0000
2,4,0.72,1,37.41,0.04,1012.02,0.66,7.97,0.0000
3,1,3.24,3,44.18,0.99,1001.50,0.89,12.40,0.2420
4,1,3.24,3,44.25,1.00,1000.81,0.90,13.70,0.1617


In [25]:
features = pd.get_dummies(X)
features.columns

Index(['day', 'distance', 'hour', 'temp', 'clouds', 'pressure', 'humidity',
       'wind', 'rain'],
      dtype='object')

In [26]:
# numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(y)

# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [27]:
train_X, test_X, train_y, test_y = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [29]:
# lets check shape of our data
print('Train_X Shape:', train_X.shape)
print('Train_y Shape:', train_y.shape)
print('Test_X Shape:', test_X.shape)
print('Test_y Shape:', test_y.shape)

Train_X Shape: (69849, 9)
Train_y Shape: (69849,)
Test_X Shape: (23283, 9)
Test_y Shape: (23283,)


In [30]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_X, train_y)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [31]:
predictions = rf.predict(test_X)
#absolute errors
errors = abs(predictions - test_y)
# mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 1.13 degrees.


In [33]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_y)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 77.53 %.
